## Pytorch

Usamos los datos de la siguiente página.

https://archive.ics.uci.edu/dataset/158/poker+hand

El conjunto describe manos en un juego de póker.  Cada mano es tomada de una baraja de 52 cartas.  Cada carta (o naipe) es descrita por dos atributos:
1. palo (_suite_): 1 - Corazón, 2 - Pica o espada, 3 - Diamante, 4 - Trébol; y
2. valor (_rank_), números del 1 al 13;
dando un total de 10 atributos.

El atributo en la última columna corresponde al tipo de mano, son números del 0 al 9 y este es el valor a predecir.

Solo se usará el archivo **poker-hand-testing.data** el cual dividiremos en los conjuntos de entrenamiento y validación.

Para instalar Pytorch puedes tu sistema operativo, tu instalador de paquetes de python y tus versiones, estaremos usando la versión 1.7.0 de pytorch, si tienes CUDA, también selecciona tu versión, en otro caso, 
puedes elegir None.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.cm as cm
plt.style.use('fivethirtyeight')

from IPython.core.pylabtools import figsize
figsize(11, 5)
colores = ["#348ABD", "#A60628","#06A628"]

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
#Definimos el dispositivo que se usará.
device = torch.device('cpu')

In [ ]:
import numpy as np

In [ ]:
def get_data(path):
    '''
    Función para obtener las matrices de entrenamiento y de validación
    a partir de los datos que se encuentran en el archivo que se define en 
    el parámetro path.
    '''
    data = np.genfromtxt(path, delimiter=',')
    X = data[:, :-1]
    y = data[:, -1]
    np.random.seed(42)
    indices = np.random.permutation(len(data))
    X = X[indices]
    y = y[indices]
    split_index = int(0.8 * len(data))
    X_train, X_val = X[:split_index], X[split_index:]
    y_train, y_val = y[:split_index], y[split_index:]
    return X_train, y_train, X_val, y_val

In [ ]:
data_path = './data/poker-hand-testing.data'
## Define una función para obtener los datos, revuelvelos 
## aleatoriamente y dividelos en los conjuntos de validación y 
## de entrenamiento.
X_train,Y_train,X_val,Y_val = get_data(data_path)

Usaremos la biblioteca nn de pytorch para definir nuestro modelo de red neuronal, puedes 
ver los tipos de capas, funciones de activación y funciones de error con las que 
disponemos viendo la documentación de pytorch. Busque la definición de 

https://pytorch.org/docs/stable/nn.html

In [ ]:
from sklearn.metrics import confusion_matrix

class Poker(nn.Module):
    def __init__(self,input_size,hidden,output_size):
        '''
        Define las caracteristicas de una red completamente conectada 
        de tres capas, recibe la cantidad de elementos de entrada, el 
        número de capas ocultas y el número de elementos de salida. 
        Entre cada capa agrega una función de activación logistica.
        '''
        super(Poker,self).__init__()
        self.fc1 = nn.Linear(input_size,hidden)
        self.fc2 = nn.Linear(hidden,hidden)
        self.fc3 = nn.Linear(hidden,output_size)

    def feed_forward(self,X):
        '''
        Define una función que de como resultado realizar la propagación
        hacia adelante de los elementos de X en la red definida.
        '''
        x= torch.sigmoid(self.fc1(X))
        x= torch.sigmoid(self.fc2(x))
        x= self.fc3(x)
        return x
    
    def back_propagate(self,X,Y):
        '''
        Define una función que realice la propagación hacia atras usando 
        la función de error de entropia cruzada.
        '''
        self.optimizer.zero_grad()
        output = self.feed_forward(X)
        m = Y.shape[0]
        loss = -torch.sum(Variable(Y) * torch.log(output) + (1 - Variable(Y)) * torch.log(1 - output)) / m
        loss.backward()
        self.optimizer.step()
        return loss.item()
        
    def train(self,train_X,train_Y,optimizer,ciclos=100):
        '''
        Define una función de entrenamiento para la red, la cual utilice
        al conjunto de entrenamiento y el algoritmo de optimización que se 
        obtenga como parametro. Al finalizar los ciclos muestra la gráfica 
        del error.
        '''
        criterio = nn.BCEWithLogitsLoss()
        losses = []
        for i in range(ciclos):
            optimizer.zero_grad()
            salida = self.feed_forward(train_X)
            perdida = criterio(salida, train_Y)
            perdida.backward()
            optimizer.step()
            losses.append(perdida.item())
            if i % 10 == 0:
                print(f'Iteración {i} - Error: {perdida.item()}')
        plt.plot(losses)
        plt.ylabel('Perdida')
        plt.title('Entrenamiento de la Red')
        plt.show()
    
    def confusion(self,test_X,test_Y):
        '''
        Muestra la matriz de confusión que presenta los valores actuales de
        la red, respecto al conjunto de datos que se decida usar.
        '''
        salidas = self.feed_forward(test_X)
        _, predicted = torch.max(salidas.data, 1)
        cm = confusion_matrix(test_Y, predicted)
        correcto = (predicted == test_Y).sum().item()
        total = predicted.size(0)
        print(f'Porcentaje de aciertos: {correcto / total * 100}')
        return cm



Usando la misma definición de la red podemos modificar fácilmente los algoritmos de optimización que 
usamos para su entrenamiento. Prueba los siguientes algoritmos de optimización con tus mismos datos 
y con la misma cantidad de iteraciones.

Las definiciones de estos algorimos las puedes ver en la documentación de pytorch

https://pytorch.org/docs/stable/optim.html

In [ ]:
## Entrena una red utilizando el algoritmo de optimización de
## stochastic gradient descent y muestra la matriz de confusión

def train_poker(hidden,ciclos,lr):
    input_size = X_train.shape[1]
    output_size = 10
    model = Poker(input_size, hidden, output_size)
    model.optimizer = optim.Adam(model.parameters(), lr=lr)
    Y_entrenamiento_int = Y_train.astype(int)
    Y_train_etiqueta = np.eye(output_size)[Y_entrenamiento_int]
    model.train(torch.tensor(X_train).float(), torch.tensor(Y_train_etiqueta).float(), model.optimizer, ciclos)
input_size = X_train.shape[1]
output_size = 10
hidden = 10
model = Poker(input_size, hidden, output_size)
model.optimizer = optim.Adam(model.parameters(), lr=0.001)
Y_entrenamiento_int = Y_train.astype(int)
Y_train_etiqueta = np.eye(output_size)[Y_entrenamiento_int]
model.train(torch.tensor(X_train).float(), torch.tensor(Y_train_etiqueta).float(), model.optimizer, 100)
print(model.confusion(torch.tensor(X_val).float(), torch.tensor(Y_val).long()))



In [ ]:
## Entrena una red utilizando el algoritmo de optimización Adam
## y muestra la matriz de confusión

def entrenar_con_Adam(hidden,ciclos,lr):
    input_size = X_train.shape[1]
    output_size = 10
    model = Poker(input_size, hidden, output_size)
    model.optimizer = optim.Adam(model.parameters(), lr=lr)
    Y_entrenamiento_int = Y_train.astype(int)
    Y_train_etiqueta = np.eye(output_size)[Y_entrenamiento_int]
    model.train(torch.tensor(X_train).float(), torch.tensor(Y_train_etiqueta).float(), model.optimizer, ciclos)
    print(model.confusion(torch.tensor(X_val).float(), torch.tensor(Y_val).long()))

interact(entrenar_con_Adam, hidden=(1, 100, 1), ciclos=(1, 100, 1), lr=(0.0001, 0.1, 0.0001))



In [ ]:
## Entrena una red utilizando el algoritmo de optmización Adagrad
## y muestra la matriz de confusión.

def entrenar_con_Adagrad(hidden,ciclos,lr):
    input_size = X_train.shape[1]
    output_size = 10
    model = Poker(input_size, hidden, output_size)
    model.optimizer = optim.Adagrad(model.parameters(), lr=lr)
    Y_entrenamiento_int = Y_train.astype(int)
    Y_train_etiqueta = np.eye(output_size)[Y_entrenamiento_int]
    model.train(torch.tensor(X_train).float(), torch.tensor(Y_train_etiqueta).float(), model.optimizer, ciclos)
    print(model.confusion(torch.tensor(X_val).float(), torch.tensor(Y_val).long()))

interact(entrenar_con_Adagrad, hidden=(1, 100, 1), ciclos=(1, 100, 1), lr=(0.0001, 0.1, 0.0001))

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read() #or edit path to custom.css
    return HTML(styles)
css_styling()